In [1]:
import numpy as np
from scipy.stats import skew
from scipy.stats import entropy
import pandas as pd

In [2]:
# CSV 파일 읽기
df = pd.read_csv('삼성전자 주식데이터2.csv', encoding='cp949')
close = df['종가'].to_numpy(dtype=np.float64)
volumes = df['거래량'].to_numpy(dtype=np.float64)
open = df['시가'].to_numpy(dtype=np.float64)
high = df['고가'].to_numpy(dtype=np.float64)
low = df['저가'].to_numpy(dtype=np.float64)
market_cap = df['시가총액'].to_numpy(dtype=np.float64)
shares_outstanding = df['상장주식수'].to_numpy(dtype=np.float64)
up_down = df['결과'].to_numpy(dtype=np.float64)     # 예측 결과값 배열


In [3]:
# 변수 선언
width = 20  # 특성 계산을 위한 윈도우 크기 설정

result = []  # 결과를 저장할 리스트

for i in range(0, len(df)-width-1):
    close_width = close[i:i+width]  # 종가 데이터
    volumes_width = volumes[i:i+width]  # 거래량 데이터
    open_width = open[i:i+width]  # 시가 데이터
    high_width = high[i:i+width]  # 고가 데이터
    low_width = low[i:i+width]  # 저가 데이터
    market_cap_width = market_cap[i:i+width]  # 시가총액
    shares_outstanding_width = shares_outstanding[i:i+width]  # 상장주식수
    
    # 대비 (현재 종가 - 이전 종가)
    price_diff = close_width[-1] - close_width[-2]
    
    # 등락률 (현재 종가와 이전 종가 비율)
    price_change = (close_width[-1] / close_width[-2]) - 1
    
    result.append([
        # 5일 이동평균
        np.mean(close_width[-5:]),
        
        # 20일 이동평균
        np.mean(close_width),
        
        # 5일 표준편차
        np.std(close_width[-5:]),
        
        # 5일 이동평균 대비 현재 종가 비율
        close_width[-1] / np.mean(close_width[-5:]),
        
        # 5일 이동평균 대비 현재 거래량 비율
        volumes_width[-1] / np.mean(volumes_width[-5:]),
        
        # 볼린저 밴드 위치
        (close_width[-1] - (np.mean(close_width) - 2 * np.std(close_width))) / (4 * np.std(close_width)),
        
        # Z-score : 표준화 점수
        (close_width[-1] - np.mean(close_width[-5:])) / np.std(close_width[-5:]),
        
        # 가격 거래량 비율
        np.mean(np.abs(np.diff(close_width)[-5:])) / np.mean(volumes_width[-5:]),
        
        # 가격 가속도
        (close_width[-1] - close_width[-2]) - (close_width[-2] - close_width[-3]),
        
        # 추세 강도
        sum(1 for i in range(1, len(close_width)) if close_width[i] > close_width[i-1]) / (len(close_width)-1),
        
        # 시가 대비 종가 변화율
        (close_width[-1] - open_width[-1]) / open_width[-1],
        
        # 고가 대비 저가 변화율
        (high_width[-1] - low_width[-1]) / open_width[-1],
        
        # 시가 대비 종가 비율
        close_width[-1] / open_width[-1],
        
        # 고가 대비 저가 차이
        high_width[-1] - low_width[-1],
        
        # 시가총액 대비 거래대금 비율
        np.mean(volumes_width) / np.mean(market_cap_width),
        
        # 상장주식수 대비 거래량 비율
        np.mean(volumes_width) / np.mean(shares_outstanding_width),
        
        # 평균 거래량 대비 변화율
        (volumes_width[-1] - np.mean(volumes_width)) / np.mean(volumes_width),
        
        # 고가 대비 저가 비율
        low_width[-1] / high_width[-1],

        # 일별 종가 대비 시가 비율 (시가: open, 종가: close)
        close_width[-1] / open_width[-1],

        # 최근 5일 이동평균 대비 고가 비율
        high_width[-1] / np.mean(close_width[-5:]),
        
        # 최근 5일 이동평균 대비 저가 비율
        low_width[-1] / np.mean(close_width[-5:]),
        
        # 5일 거래량 변화율
        (volumes_width[-1] - volumes_width[-5]) / volumes_width[-5],
        
        # 일별 고가-저가 비율
        (high_width[-1] - low_width[-1]) / open_width[-1],
        
        # 5일 고가-저가 차이의 표준편차
        np.std(np.array(high_width[-5:]) - np.array(low_width[-5:])),
        
        # 추세 전환 점수 (예시: 5일 이동평균 크로스오버)
        np.mean(np.diff(close_width[-5:])) / np.mean(close_width[-5:]),
        
        # 가격 변동성 (5일)
        np.std(np.diff(close_width[-5:])),
        
        # 등락률에 따른 변화
        price_change,
        
        up_down[i+width]
    ])

In [4]:
df4_result = pd.DataFrame(result, columns=['5일 이동평균', '20일 이동평균', '5일 표준편차', '5일 이동평균 대비 현재 종가 비율',
                                           '5일 이동평균 대비 현재 거래량 비율', '볼린저 밴드 위치', 'Z-score', '가격 거래량 비율',
                                           '가격 가속도', '추세 강도', '시가 대비 종가 변화율', '고가 대비 저가 변화율',
                                           '시가 대비 종가 비율', '고가 대비 저가 차이', '시가총액 대비 거래 대금 비율', '상장주식수 대비 거래량 비율',
                                           '평균 거래량 대비 변화율', '고가 대비 저가 비율', '일별 종가 대비 시가 비율', '최근 5일 이동 평균 대비 고가 비율',
                                           '최근 5일 이동평균 대비 저가 비율', '5일 거래량 변화율', '일별 고가-저가 비율',
                                           '5일 고가 - 저가 차이의 표준편차', '추세 전환 점수', '가격 변동성', '등락률에 따른 변화',
                                           '예측 결과값'])

df4_result.to_csv('result_A.csv', index=False, header=False, encoding='cp949')

data = np.vstack(df4_result.values.astype(float))
np.random.seed(42)
np.random.shuffle(data)

total_rows = data.shape[0]
split_index = int(total_rows * 2 / 3)

traindata = data[:split_index, :]
testdata = data[split_index:, :]

np.savetxt('result_train.csv', traindata, delimiter=',', fmt='%f', encoding='utf-8')
np.savetxt('result_test.csv', testdata, delimiter=',', fmt='%f', encoding='utf-8')

In [14]:
import numpy as np
import keras
import matplotlib.pyplot as plt
from keras import models
from keras import layers
from sklearn.metrics import accuracy_score
from keras.callbacks import EarlyStopping
from keras.layers import Dropout
from sklearn.preprocessing import StandardScaler

traindata = np.loadtxt('result_train.csv', delimiter=',', dtype=np.float32)
train_images = traindata[:,0:-1]
train_labels = traindata[:,[-1]]

testdata = np.loadtxt('result_test.csv', delimiter=',', dtype=np.float32)
test_images = testdata[:,0:-1]
test_labels = testdata[:,[-1]]

scaler = StandardScaler()
train_images_scaled = scaler.fit_transform(train_images)
test_images_scaled = scaler.transform(test_images)

In [16]:
train_images_scaled.shape

(298, 27)

In [18]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.layers import MultiHeadAttention, LayerNormalization, Dense, Dropout

# Transformer 블록 정의
class TransformerBlock(layers.Layer):
    def __init__(self, num_heads, ff_dim, dropout_rate=0.1):
        super(TransformerBlock, self).__init__()
        self.attention = MultiHeadAttention(num_heads=num_heads, key_dim=ff_dim)
        self.ffn = models.Sequential([
            Dense(ff_dim, activation="relu"),
            Dense(ff_dim)
        ])
        self.layer_norm1 = LayerNormalization()
        self.layer_norm2 = LayerNormalization()
        self.dropout = Dropout(dropout_rate)

    def call(self, inputs, training=None):
        # Multi-Head Attention
        attention_output = self.attention(inputs, inputs)
        attention_output = self.dropout(attention_output, training=training)
        attention_output = self.layer_norm1(inputs + attention_output)

        # Feed Forward Network
        ffn_output = self.ffn(attention_output)
        ffn_output = self.dropout(ffn_output, training=training)
        output = self.layer_norm2(attention_output + ffn_output)

        return output




In [ ]:
accuracies = []
train_accuracies = []
val_accuracies = []
for _ in range(5):
    # 모델 정의
    model = models.Sequential()
    
    # 입력 형태에 맞게 Reshape
    model.add(layers.Input(shape=(27, 1)))  # (batch_size, seq_len, feature_dim) 형태로 입력
    
    # Transformer Block 추가
    model.add(TransformerBlock(num_heads=4, ff_dim=64, dropout_rate=0.1))
    
    # 출력 부분
    model.add(layers.Flatten())  # Transformer의 출력을 평탄화
    model.add(layers.Dense(64, activation='relu'))  # 추가적인 Dense 레이어
    model.add(layers.Dense(1, activation='sigmoid'))  # 이진 분류 출력 (예: 0 또는 1)
    
    # 모델 컴파일
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

    # 모델 학습
    history = model.fit(train_images_scaled, train_labels,
                          batch_size=64,  # 배치 크기 변경
                          epochs=100,
                          validation_data=(test_images_scaled, test_labels),
                          verbose=1,
                          shuffle=True)

    # 예측
    predicted_probs = model.predict(test_images_scaled)  # 확률 예측값 (0과 1 사이)
    
    # 예측 결과를 이진 값으로 변환
    predicted_labels = (predicted_probs > 0.5).astype(int)

    # 정확도 계산
    accuracy = accuracy_score(test_labels, predicted_labels)
    train_accuracies.append(history.history['accuracy'])
    val_accuracies.append(history.history['val_accuracy'])

    print(f"정확도: {accuracy:.4f}")
    
    accuracies.append(accuracy)



Epoch 1/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 6s 204ms/step - accuracy: 0.4998 - loss: 0.6931 - val_accuracy: 0.4933 - val_loss: 0.6932
Epoch 2/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 26ms/step - accuracy: 0.4878 - loss: 0.6932 - val_accuracy: 0.4933 - val_loss: 0.6932
Epoch 3/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step - accuracy: 0.5151 - loss: 0.6931 - val_accuracy: 0.4933 - val_loss: 0.6932
Epoch 4/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step - accuracy: 0.5353 - loss: 0.6930 - val_accuracy: 0.4933 - val_loss: 0.6932
Epoch 5/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step - accuracy: 0.5281 - loss: 0.6930 - val_accuracy: 0.4933 - val_loss: 0.6932
Epoch 6/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step - accuracy: 0.5268 - loss: 0.6930 - val_accuracy: 0.4933 - val_loss: 0.6932
Epoch 7/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step - accuracy: 0.5108 - loss: 0.6931 - val_accuracy: 0.4933 - val_loss: 0.6932
Epoch 8/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step - accuracy: 0.5145 - loss: 0.6930 - val_accuracy: 0.4933 - val_los

In [ ]:
max_length_train = max(len(acc) for acc in train_accuracies)
max_length_val = max(len(acc) for acc in val_accuracies)

train_accuracies_padded = [np.pad(acc, (0, max_length_train - len(acc)), 'constant') for acc in train_accuracies]
val_accuracies_padded = [np.pad(acc, (0, max_length_val - len(acc)), 'constant') for acc in val_accuracies]

train_accuracies_array = np.array(train_accuracies_padded)
val_accuracies_array = np.array(val_accuracies_padded)

mean_train_accuracy = train_accuracies_array.mean(axis=0)
mean_val_accuracy = val_accuracies_array.mean(axis=0)

plt.figure(figsize=(12, 4))
plt.plot(mean_train_accuracy, label='Average Training Accuracy', color='blue')
plt.plot(mean_val_accuracy, label='Average Validation Accuracy', color='orange')
plt.title('Average Model Accuracy Across Runs')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.grid()
plt.show()

In [ ]:
accuracies = []
train_accuracies = []
val_accuracies = []

for _ in range(5):
    network = models.Sequential()
    network.add(layers.Dense(64, activation='relu', input_shape=(27,)))  # 27개의 입력 특성
    network.add(layers.Dense(32, activation='relu'))
    network.add(layers.Dense(1, activation='sigmoid'))
    network.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])
    
    
    history = network.fit(train_images_scaled, train_labels,
                          batch_size=32,
                          epochs=100,
                          verbose=1,
                          shuffle='batch',
                          validation_data=(test_images_scaled, test_labels))

    predict = network.predict(test_images_scaled)

    predicted_labels = (predict > 0.5).astype(int)

    accuracy = accuracy_score(test_labels, predicted_labels)
    train_accuracies.append(history.history['accuracy'])
    val_accuracies.append(history.history['val_accuracy'])

    print(f"정확도: {accuracy:.4f}")
    
    accuracies.append(accuracy)